In [ ]:
Install dependencies fast.

In [ ]:
!pip install python-dotenv google-cloud-aiplatform matplotlib z3-solver

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("config.env")

VECTOR_AI_KEY = os.getenv("VECTOR_AI_KEY")
VECTOR_AI_PROJECT = os.getenv("VECTOR_AI_PROJECT")
VECTOR_AI_MODEL = os.getenv("VECTOR_AI_MODEL")

GCP_PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
GCP_LOCATION   = os.getenv("GOOGLE_CLOUD_REGION")

print("Vector key loaded:", VECTOR_AI_KEY is not None)
print("GCP Project:", GCP_PROJECT_ID)


In [4]:
from vertexai import init as vertex_init

vertex_init(project=GCP_PROJECT_ID, location=GCP_LOCATION)
print("Vertex AI initialized.")


In [ ]:
from vertexai.preview import generative_models as gemini

model = gemini.GenerativeModel("gemini-1.5-pro")


In [ ]:
from z3 import *

class RoomConstraintSolver:
    def __init__(self, width, height, n_chairs=4):
        self.width = width
        self.height = height
        self.n_chairs = n_chairs

        # Z3 Variables
        self.table_x = Real("table_x")
        self.table_y = Real("table_y")

        self.focal_x = Real("focal_x")
        self.focal_y = Real("focal_y")

        self.chairs = [
            (Real(f"chair_{i}_x"), Real(f"chair_{i}_y"))
            for i in range(n_chairs)
        ]

        self.solver = Solver()
        self.apply_constraints()

    def apply_constraints(self):

        # Table inside room
        self.solver.add(self.table_x > 1, self.table_x < self.width-1)
        self.solver.add(self.table_y > 1, self.table_y < self.height-1)

        # Focal point on one side of the room
        self.solver.add(self.focal_x == self.width / 2)
        self.solver.add(self.focal_y == self.height - 0.5)

        # Constraints for each chair
        for (cx, cy) in self.chairs:
            # inside room
            self.solver.add(cx > 0.5, cx < self.width-0.5)
            self.solver.add(cy > 0.5, cy < self.height-1)

            # walkway: at least 1m from table
            self.solver.add(Abs(cx - self.table_x) + Abs(cy - self.table_y) >= 1)

            # cannot block focal point
            self.solver.add(cy < self.focal_y - 0.5)

    def solve(self):
        if self.solver.check() == sat:
            m = self.solver.model()
            return {
                "table": {"x": float(m[self.table_x].as_decimal(6)), 
                          "y": float(m[self.table_y].as_decimal(6))},
                "focal_point": {"x": float(m[self.focal_x].as_decimal(6)),
                                "y": float(m[self.focal_y].as_decimal(6))},
                "chairs": [
                    {"x": float(m[cx].as_decimal(6)), 
                     "y": float(m[cy].as_decimal(6))}
                    for (cx, cy) in self.chairs
                ]
            }
        else:
            return None


In [ ]:
class RoomOptimizer:

    def __init__(self):
        self.llm = llm
        self.vector_key = VECTOR_AI_KEY

    def optimize(self, size="6x6", n_chairs=4):
        w, h = map(int, size.split("x"))

        # Step 1 — Logic reasoning (symbolic)
        solver = RoomConstraintSolver(w, h, n_chairs)
        layout = solver.solve()

        # Step 2 — Vector query (stub)
        vector_summary = {
            "vector_key_valid": bool(self.vector_key),
            "query": f"room layout {size}"
        }

        # Step 3 — LLM refinements/description
        prompt = f"""
Here is a room layout produced by a strict constraint system:

LAYOUT:
{layout}

VECTOR SUMMARY:
{vector_summary}

TASK:
Write a short explanation of the layout decisions and
provide improved naming, descriptions, and arrangement ideas
while preserving the coordinates exactly as they are.

DO NOT modify the coordinates.
"""

        explanation = self.llm.generate_content(prompt).text
        return layout, explanation


In [ ]:
optimizer = RoomOptimizer()

layout, explanation = optimizer.optimize("6x6", n_chairs=4)

print("LOGIC-GENERATED LAYOUT:")
layout


In [ ]:
print(explanation)


In [ ]:
import matplotlib.pyplot as plt

def plot_room(layout, room_w=6, room_h=6):
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xlim(0, room_w)
    ax.set_ylim(0, room_h)
    ax.set_aspect("equal")
    ax.set_title("Room Layout (Logic Constrained)")

    # Plot table
    t = layout["table"]
    plt.scatter(float(t["x"]), float(t["y"]), s=600, label="Table")

    # Plot chairs
    for c in layout["chairs"]:
        plt.scatter(float(c["x"]), float(c["y"]), marker="s", s=200, label="Chair")

    # Plot focal point
    f = layout["focal_point"]
    plt.scatter(float(f["x"]), float(f["y"]), marker="^", s=300, label="Focal Point")

    plt.legend()
    plt.grid(True)
    plt.show()

plot_room(layout)
